In [2]:
#
#Tweets Machine Learning Coursework 
#COMP3222
#

# To support both python 2 and 3
from __future__ import division, print_function, unicode_literals

# Common imports
import os
import numpy as np
import pandas as pd

# These two lines are required to use Tensorflow 1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# To plot nice figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Clear tensorflow's and reset seed
def reset_graph(seed=None):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)



Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
#get data
def getData(file_name):
    dict_data = {}
    data_file = open(file_name, "r", encoding="utf8")
    raw_data_txt = data_file.readlines()
    raw_data = []
    for data in raw_data_txt:
        raw_data.append(data.split("\t"))
    raw_data.pop(0)

    target_text = [data[6] for data in raw_data]
    dict_data['tweetId'] = np.array([tweet[0] for tweet in raw_data])
    dict_data['text'] = np.array([tweet[1] for tweet in raw_data])
    dict_data['userId'] = np.array([tweet[2] for tweet in raw_data])
    dict_data['imageIds'] = np.array([tweet[3] for tweet in raw_data])
    dict_data['username'] = np.array([tweet[4] for tweet in raw_data])
    dict_data['timestamp'] = np.array([tweet[5] for tweet in raw_data])
    dict_data['label'] = np.array([1 if target=='real' or target =='real\n' else 0 for target in target_text])

    return dict_data

df_train = pd.DataFrame.from_dict(getData("mediaeval-2015-trainingset.txt"))
df_test = pd.DataFrame.from_dict(getData("mediaeval-2015-testset.txt"))
print("Done")


Done


In [80]:
def scan_t_id(df):
    check = ['.', ]
    for i in range(len(df)):
        x = str(df.iloc[i]['username'])
        if(x.count('.')>0):
            print(x)
        

scan_t_id(df_train)
#scan_t_id(df_test)



In [65]:
df_train = pd.read_csv("train_v2.csv")
df_train = df_train.iloc[:, 1:]
df_train
df_train.text = df_train.text.astype(str)

In [98]:
def date(df):
    for i in range(len(df)):
        x = str(df.iloc[i]['timestamp'])
        x = x.split(' ')
        if(len(x) != 6):
            print(i, x)
        #print(x[3][3:5])
date(df_train)
# date(df_test)

14460 ['Tue', 'Mar', '11', '23:', '59:', '22', '+0000', '2014']
14461 ['Tue', 'Mar', '11', '13:', '57:', '22', '+0000', '2014']
14462 ['Tue', 'Mar', '11', '03:', '13:', '40', '+0000', '2014']
14463 ['Tue', 'Mar', '11', '03:', '13:', '40', '+0000', '2014']
14464 ['Mon', 'Mar', '10', '22:', '00:', '13', '+0000', '2014']
14465 ['Mon', 'Mar', '10', '21:', '47:', '50', '+0000', '2014']
14466 ['Mon', 'Mar', '10', '21:', '21:', '02', '+0000', '2014']
14467 ['Mon', 'Mar', '10', '19:', '32:', '38', '+0000', '2014']
14468 ['Mon', 'Mar', '10', '14:', '11:', '23', '+0000', '2014']
14469 ['Mon', 'Mar', '17', '11:', '46:', '36', '+0000', '2014']
14470 ['Mon', 'Mar', '17', '08:', '54:', '26', '+0000', '2014']
14471 ['Sun', 'Mar', '16', '03:', '48:', '40', '+0000', '2014']
14472 ['Fri', 'Mar', '14', '17:', '31:', '06', '+0000', '2014']
14473 ['Thu', 'Mar', '13', '20:', '07:', '05', '+0000', '2014']
14474 ['Thu', 'Mar', '13', '02:', '37:', '05', '+0000', '2014']
14475 ['Thu', 'Mar', '13', '00:', '15:',

In [97]:
import regex as re

x = "My cousin sent this to me... :: Cleveland voice ::     We gon' die!!! ☁⚡☔🌊 #hurricane #hurricanesandy #sandy http://t.co/XtW4exLn"
parsed = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
parsed.lower()

'my cousin sent this to me cleveland voice we gon die hurricane hurricanesandy sandy'

In [70]:
#Spell checker test code
from spellchecker import SpellChecker

spell = SpellChecker()
for i in range(len(df_train)):
    test = df_train.iloc[i]['text']
    test = test.split(" ")

    # find those words that may be misspelled
    misspelled_test = spell.unknown(test)
    misspelling_found_test = False
    misspell_count_test = 0
    corrections = []
    for word in misspelled_test:
        corrections.append(spell.correction(word))
    if(len(misspelled_test)>0):
        print(i, misspelled_test, corrections)


# print(len(misspelled_test))
# print(test)
# print(misspelled_test)

1 {'rt'} ['it']
2 {'id4'} ['id']
4 {'nyc', 'statueofliberty'} ['ny', 'statueofliberty']
5 {'nyc', '42nd'} ['ny', 'and']
6 {'frankenstorm'} ['frankenstorm']
8 {'newyork', 'statueofliberty'} ['network', 'statueofliberty']
9 {'nyc'} ['ny']
10 {'newyork', 'robertosalibaba'} ['network', 'robertosalibaba']
12 {'newjersey'} ['newjersey']
13 {'newyork'} ['network']
16 {'newyork', 'bcs'} ['network', 'bus']
17 {'nh', 'md', 'ct', 'nc', 'nj'} ['no', 'my', 'it', 'no', 'no']
18 {'nyc', 'newyork'} ['ny', 'network']
20 {'hurricanesandy', 'statueofliberty'} ['hurricanesandy', 'statueofliberty']
22 {'hurricanesandy'} ['hurricanesandy']
23 {'frankenstorm'} ['frankenstorm']
25 {'newyork', 'picoftheday', 'statueofliberty'} ['network', 'picoftheday', 'statueofliberty']
26 {'puupy'} ['puppy']
27 {'nyc'} ['ny']
28 {'nyc', 'jerseyithink', 'naturaldisasterornot', 'somethingisntright'} ['ny', 'jerseyithink', 'naturaldisasterornot', 'somethingisntright']
30 {'nyc'} ['ny']
32 {'nodaysoff', 'dayoff'} ['nodaysoff', 

KeyboardInterrupt: 

In [83]:
x = str("Liver un facebook para cuando andas pedo http: \/\/t.co\/1sivd33y1j http: \/\/t.co\/cP63lVuAz8")
parsed = ' '.join(re.sub("(@[A-Za-z0-9_]+)|((\\\/\\\/\w+.co\S+)|(\w+:\/\/\S+))|([^0-9A-Za-z \t])"," ",x).split())
parsed = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split())
print(parsed)

Liver un facebook para cuando andas pedo http t co 1sivd33y1j http t co cP63lVuAz8


In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer

x = "Hello my name is Michael and the weather is very fine"
y = "Hello the word is Michael and the weather Hello"

v = TfidfVectorizer()
response = v.fit_transform([x, y])
print(response)



  (0, 1)	0.34186893933428164
  (0, 8)	0.34186893933428164
  (0, 9)	0.24324256601037686
  (0, 7)	0.24324256601037686
  (0, 0)	0.24324256601037686
  (0, 4)	0.24324256601037686
  (0, 3)	0.4864851320207537
  (0, 6)	0.34186893933428164
  (0, 5)	0.34186893933428164
  (0, 2)	0.24324256601037686
  (1, 10)	0.3759577127469508
  (1, 9)	0.26749700905276197
  (1, 7)	0.5349940181055239
  (1, 0)	0.26749700905276197
  (1, 4)	0.26749700905276197
  (1, 3)	0.26749700905276197
  (1, 2)	0.5349940181055239


In [92]:
print(v.get_feature_names())
token_id = list

['and', 'fine', 'hello', 'is', 'michael', 'my', 'name', 'the', 'very', 'weather', 'word']


In [6]:
df_train = pd.read_csv("train_v2.2.csv")
df_train = df_train.iloc[:, 1:]
df_train
df_train.text = df_train.text.astype(str)
df_train.pos = df_train.pos.astype(str)

In [17]:
from collections import defaultdict

POS_tags_fake = defaultdict(lambda: 0)
POS_tags_real= defaultdict(lambda: 0)
POS_tags_total= defaultdict(lambda: 0)

for i in range(len(df_train)):
    text = str(df_train.iloc[i]['pos'])
    text = text.split()
    if(df_train.iloc[i]['label'] == 0):
        for j in range(len(text)):
            POS_tags_fake[text[j]] += 1
            POS_tags_total[text[j]] += 1
    else:
        for j in range(len(text)):
            POS_tags_real[text[j]] += 1
            POS_tags_total[text[j]] += 1


In [18]:
POS_tags_fake = dict(sorted(POS_tags_fake.items(), reverse=True, key=lambda item: item[1]))
POS_tags_real = dict(sorted(POS_tags_real.items(), reverse=True, key=lambda item: item[1]))
POS_tags_total = dict(sorted(POS_tags_total.items(), reverse=False, key=lambda item: item[1]))

In [22]:
from collections import Counter
rw = Counter(POS_tags_real)
fw = Counter(POS_tags_fake)
# fw = fw - rw
# rw = rw - fw
POS_tags_total = {key:val for key, val in POS_tags_total.items() if val > 500}
rw = {k: rw[k]/POS_tags_total[k] for k in POS_tags_total.keys() & rw}
fw = {k: fw[k]/POS_tags_total[k] for k in POS_tags_total.keys() & fw}

rw = dict(sorted(rw.items(), reverse=True, key=lambda item: item[1]))
fw = dict(sorted(fw.items(), reverse=True, key=lambda item: item[1]))
fw, rw

({'VBD': 0.6552714737144498,
  'NNS': 0.6433876221498371,
  'RB': 0.6416107382550336,
  'VB': 0.6401617250673854,
  'VBZ': 0.636930754834685,
  'JJ': 0.6333138079769955,
  'NN': 0.6290381052122774,
  'VBN': 0.6207928197456993,
  'VBP': 0.6192210703278229,
  'VBG': 0.6023040320560982,
  'IN': 0.5735981308411215,
  'CD': 0.5361179361179361},
 {'CD': 0.4638820638820639,
  'IN': 0.4264018691588785,
  'VBG': 0.39769596794390183,
  'VBP': 0.3807789296721771,
  'VBN': 0.37920718025430067,
  'NN': 0.37096189478772257,
  'JJ': 0.3666861920230044,
  'VBZ': 0.363069245165315,
  'VB': 0.35983827493261455,
  'RB': 0.35838926174496644,
  'NNS': 0.35661237785016286,
  'VBD': 0.3447285262855501})

In [21]:
POS_tags_total['FW']

167

In [5]:
from nltk import ngrams

def n_grams(data_frame, n):
    a = []
    for i in range(len(data_frame)):
        sentence = data_frame.iloc[i]['text']

        ng = ngrams(sentence.split(), n)
        x = []
        for grams in ng:
            x.append(grams)
        a.append(x)
    data_frame.insert(3, ng+"-gram", a, True)

('this', 'is', 'this', 'foo', 'bar', 'sentences')
('is', 'this', 'foo', 'bar', 'sentences', 'and')
('this', 'foo', 'bar', 'sentences', 'and', 'i')
('foo', 'bar', 'sentences', 'and', 'i', 'want')
('bar', 'sentences', 'and', 'i', 'want', 'to')
('sentences', 'and', 'i', 'want', 'to', 'ngramize')
('and', 'i', 'want', 'to', 'ngramize', 'it')
